# Text classification

[Project requirements](https://review.udacity.com/#!/rubrics/1507/view)

# Open Data Set for Text classification selection

After searching for some Text classification Open Data Set detailed below, I chose [XX](XX)

[Open Data Sets for text classification](https://medium.com/@dataturks/rare-text-classification-open-datasets-9d340c8c508e)

[IMBD DataSet for Movies Reviews](http://ai.stanford.edu/~amaas/data/sentiment/)

[BBC text categorization](https://www.kaggle.com/yufengdev/bbc-text-categorization)

[Sentiment Analysis in Finance](https://www.kaggle.com/ankurzing/sentiment-analysis-for-financial-news)

# Usefull inspiration when writing my medium Posts

While writing one of my first Medium posts, I found the below links very usefull :
- [Get Started with medium](https://blog.hubspot.com/marketing/how-to-use-medium)
- [Tips for medium writer](https://blog.medium.com/tips-and-tricks-for-medium-writers-1d79498101c3)

Besides, below are some Medium posts that I've found inspiring: 
- https://medium.com/@rchang
- https://medium.com/@josh_2774/how-do-you-become-a-developer-5ef1c1c68711
- https://medium.com/analytics-vidhya/how-to-succeed-in-multilingual-sentiment-analysis-without-transformers-f1a98c76c30c
- https://towardsdatascience.com/text-classification-with-extremely-small-datasets-333d322caee2

# Import Usefull Packages

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import time

[nltk_data] Downloading package punkt to /Users/boubou/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boubou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/boubou/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

# Data Exploration

In [3]:
# Load Data
data = pd.read_csv('Data/all-data.csv', 
encoding='latin-1', names=['sentiment', 'text'])
data.head(10)

,sentiment,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
5,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
6,positive,"For the last quarter of 2010 , Componenta 's n..."
7,positive,"In the third quarter of 2010 , net sales incre..."
8,positive,Operating profit rose to EUR 13.1 mn from EUR ...
9,positive,"Operating profit totalled EUR 21.1 mn , up fro..."


# Standard Approach - Tf-idf

# More advanced approach

## 1. Leverage on pre-trained Embeddings 

[Example of text classification on Keras](https://www.tensorflow.org/tutorials/keras/text_classification)

## 2. Combine LSTM with pre-trained Embeddings

[Example of text classification with RNN](https://www.tensorflow.org/tutorials/text/text_classification_rnn)